In [22]:
import json
from datetime import date,datetime
from random import randint, random as rnd
from json import dump

from faker import Faker
from pymongo import MongoClient
from pypinyin import Style, lazy_pinyin

In [2]:
client = MongoClient(host="127.0.0.1", port=27017)

In [5]:
db = client.get_database("fabric")
clothes = db.get_collection("clothes")
clients = db.get_collection("clients")
bills = db.get_collection("bills")

In [4]:
cl = []

cursor = clothes.find({})

for c in cursor:
    cl.append(c)


with open("o.json", "w") as f:
    json.dump(cl, f,ensure_ascii=False)


In [4]:
faker = Faker(locale="zh_CN")

In [12]:
l = []

for _ in range(100):
    name = faker.name()
    py = " ".join(lazy_pinyin(name, style=Style.NORMAL))

    random = {
        "_id": name,
        "pinyin": py,
        "phone": faker.phone_number(),
        "address": faker.address()[6:],
        "temp": True
    }
    l.append(random)

In [14]:
color_list = [
    "水绿", "湖蓝", "鱼肚白", "灰色", "铜绿",
    "银白", "赤金", "荼白", "丁香色", "大红",
    "茜色", "洋红", "酡红", "橘黄", "姜黄", "栗色",
    "绾", "酡颜", "鹅黄", "黄栌", "碧色", "翡翠色",
    "鸭卵青", "蟹壳青", "鸦青", "艾绿", "松花色",
    "碧蓝", "黛螺", "藕荷色", "莹白", "缃色"
]

In [16]:
cloth_list = [
    "麻", "桑蚕丝", "粘胶", "醋酯纤维", "涤纶",
    "锦纶", "丙纶", "氨纶", "夏布", "交织麻织物",
    "派力司", "华达呢", "薄花呢", "松轻毛织物",
    "麦尔登", "长毛绒", "平布", "细布", "府绸",
    "巴百纱", "卡其", "贡缎", "绉布", "牛津布",
    "丹宁", "灯芯绒", "绒布", "山羊绒", "兔毛",
    "马海毛", "骆驼毛绒", "精纺毛织物", "粗纺毛织物",
    "缎", "麻棉混纺", "涤麻混纺", "进口长绒棉",
    "绢丝", "真丝", "天丝", "丝光棉纱线", "竹纤维",
    "腈纶", "丝光羊毛", "毛粘混纺", "天丝、亚麻混纺",
    "涤棉混纺", "涤绢混纺", "涤粘华达呢"
]
unit_list = ["米", "卷", "匹", "公斤", "码"]

c = []

for i in range(300):
    idx1 = i % len(color_list)
    idx2 = i % len(cloth_list)
    idx3 = i % len(unit_list)
    name = color_list[idx1] + cloth_list[idx2]
    py = " ".join(lazy_pinyin(name))
    tmp = {
        "_id": name,
        "pinyin": py,
        "color": color_list[idx1],
        "unit": unit_list[idx3],
        "temp": True
    }

    c.append(tmp)

clothes.drop()
clothes.insert_many(c)

In [17]:
clerks = [
    "唐思粤", "朱之杰", "宏大",
    "测试店员1", "测试店员2", "测试店员3",
    "测试店员4", "测试店员5", "测试店员6"
]
client_list = clients.find()
client_list = list(map(lambda x: x["_id"], list(client_list)))
cloth_list = list(clothes.find())

In [23]:
b = []
dm = {}

def format_id(s):
    if s <10:
        return f"00{s}"
    elif s < 100:
        return  f"0{s}"
    else:
        return  str(s)


start = date(2022, 1, 1)
for i in range(40000):
    t = faker.date_time_between(start_date=start)
    d = faker.date_between(start_date=start,end_date=t)
    d = d.strftime("%Y%m%d")
    t = t.timestamp()*1000
    if d in dm.keys():
        dm[d] += 1
    else:
        dm[d] = 1
    _id = d + format_id(dm[d])

    count = faker.random_digit()
    if count <=0:
        continue

    items = []
    total = 0
    for _ in range(count):
        tmp = cloth_list[randint(0, len(cloth_list) -1)]
        price = randint(1, 120)
        c = faker.random_digit()
        total += price * c
        items.append( {
                "name": tmp["_id"],
                "color": color_list[randint(0, len(color_list) -1)],
                "unit": tmp["unit"],
                "count": c,
                "price": price,
                "money": round(c * price)
            })

    b.append({
        "_id": _id,
        "date": d,
        "clerk": clerks[randint(0, len(clerks) - 1)],
        "client": client_list[randint(0, len(client_list) - 1)],
        "items": items,
        "money": round(total),
        "received": round(total * rnd()),
        "isReturn": False,
        "createAt": t,
        "temp": True
    })

In [24]:
bills.drop()
bills.insert_many(b)

In [9]:
d =faker.date_time()
d

datetime.datetime(1991, 5, 18, 3, 23, 9)

In [10]:
d.timestamp()

674508189.0